In [ ]:
#deal 번호를 받아와서
#http://www.tmon.co.kr/api/detail/direct/v1/detailapi/api/detail/board/review/list/4284526834?&page=100&count=100
#reviewContents가 없을 때까지

In [ ]:
### import urllib
from bs4 import BeautifulSoup
import requests
import json
import re
import time
import tqdm
from tqdm.notebook import tqdm

def toJson(data, file_path) :
    with open(file_path, 'w', encoding='utf-8') as file :
        json.dump(data, file)
    return 

# txt_file_path_ls = ['productUrlList_간편식_냉장_냉동.txt', 'productUrlList_과일.txt']
txt_file_path_ls = ['productUrlList_간편식_냉장_냉동.txt'] #같은 폴더에 넣고 돌릴 것! -> 'review_data_{}.json'.format(txt_file_path[15:-4]으로 저장되기 때문

for txt_file_path in txt_file_path_ls :

    #productUrlList.txt에서 링크 리스트 불러오기
    r = open(txt_file_path, mode='rt', encoding='utf-8')
    productUrlList = r.readlines()

    review_data = []
    error_page_ls = []

    for prod_url in tqdm(productUrlList) :
        prod_idx = re.sub('http://www.tmon.co.kr/deal/','',prod_url)
        prod_idx = re.sub('\n','',prod_idx)
        
        page = 1
        while True: 
            time.sleep(1)
            if page%5 == 0 :
                time.sleep(10)
            url = "http://www.tmon.co.kr/api/detail/direct/v1/detailapi/api/detail/board/review/list/{}?&page={}&count=100".format(prod_idx, page)
            try :
                html = urllib.request.urlopen(url).read()
            except :
                time.sleep(120)
                html = urllib.request.urlopen(url).read()
            soup = BeautifulSoup(html, 'html.parser')
            text = soup.get_text()
#             print(text)
            try :
                json_data = json.loads(text)
            except :
#                 print("error page : {}".format(page))
                error_page_ls.append(url)
                page += 1
                continue
                
            iter_length = len(json_data['data']['reviewContents'])
            
            if iter_length == 0 :
                break
        
            for idx in range(iter_length) :

                #리뷰 고유 번호
                review_idx  = json_data['data']['reviewContents'][idx]['reviewSrl']
                #상품 번호
                prod_idx = prod_idx
                #상품 만족도
                prod_satis = json_data['data']['reviewContents'][idx]['dealGpoint']
                #배송 만족도
                ship_satis = json_data['data']['reviewContents'][idx]['deliveryGpoint']
                #구매한 옵션
                try :
                    select_options = json_data['data']['reviewContents'][idx]['buyOptionList'][0]['optionTitle']
                except :
                    select_options =''
                #구매한 날짜
                try :
                    buy_date = json_data['data']['reviewContents'][idx]['buyOptionList'][0]['buyDate']
                except :
                    buy_date = ''
                #리뷰 작성 날짜
                try :
                    review_date = json_data['data']['reviewContents'][idx]['reviewSurveyList'][0]['updateDt']
                except :
                    review_date = ''
                #사용만족도 옵션1
                try :
                    use_opt1 = json_data['data']['reviewContents'][idx]['reviewSurveyList'][0]['keyword']
                    use_satis1 = json_data['data']['reviewContents'][idx]['reviewSurveyList'][0]['selection']
                    #사용만족도 옵션2
                    use_opt2 = json_data['data']['reviewContents'][idx]['reviewSurveyList'][1]['keyword']
                    use_satis2 = json_data['data']['reviewContents'][idx]['reviewSurveyList'][1]['selection']
                except : 
                    use_opt1 = ''
                    use_satis1 = ''
                    use_opt2 = ''
                    use_satis2 = ''
                #리뷰 내용
                content = json_data['data']['reviewContents'][idx]['contents']
                #user_id
                user_id = json_data['data']['reviewContents'][idx]['whoUpdate']
                #memberSrl
                memberSrl =json_data['data']['reviewContents'][idx]['memberSrl']

                review_data.append({
                        'review_idx' : review_idx,
                        'prod_idx' : prod_idx,
                        'prod_satis' : prod_satis,
                        'ship_satis' : ship_satis,
                        'select_options' : select_options,
                        'buy_date' : buy_date,
                        'review_date' : review_date,
                        'use_opt1' : use_opt1,
                        'use_satis1' : use_satis1,
                        'use_opt2' : use_opt2,
                        'use_satis2' : use_satis2,
                        'content' : content,
                        'user_id' : user_id,
                        'memberSrl' : memberSrl
                    })
                

            page += 1

#     toJson(review_data, 'review_data_{}.json'.format(간편식_냉장_냉동))
#     toJson(error_page_ls, 'error_page_{}.json'.format(간편식_냉장_냉동))

In [5]:
toJson(review_data, 'review_data_간편식_냉장_냉동_수정.json')
toJson(error_page_ls, 'error_page_간편식_냉장_냉동_수정.json')

In [6]:
import pandas as pd
data_df = pd.read_json('review_data_간편식_냉장_냉동_수정.json')
data_df

,review_idx,prod_idx,prod_satis,ship_satis,select_options,buy_date,review_date,use_opt1,use_satis1,use_opt2,use_satis2,content,user_id,memberSrl
0,172454210,537157714,5,5,[숨김]10_갈비천왕 치밥 브리또 1팩_KM01,2020-05-14 02:59:08,2020-10-19 17:19:10,맛,맛있어요,포장,꼼꼼해요,전 매운게 더 맛있었네요 ㅎㅎ 잘먹었습니다,kal********,67017718
1,172442050,537157714,5,5,a04_훈제 슬라이스 오리지널 1팩_IA01,2020-10-08 18:15:58,2020-10-19 16:24:20,맛,맛있어요,포장,꼼꼼해요,사은품 챙겨주셔서 감사해요.이것저것담았는데.. 저나트륨은 정말 건강식의 맛이고. 마...,jic***,5414261
2,172399322,537157714,5,5,13_볼케이노 치밥 1팩_KB82,2020-10-14 23:00:28,2020-10-19 13:18:26,맛,맛있어요,포장,꼼꼼해요,볼케이노 소스 사려고 이것저것 담았는데 굽네 맛있어요,yew*****,3141293
3,172304958,537157714,4,5,[숨김]23_소스가 맛있는 닭가슴살 슬라이스 레드크림 커리 1팩_GC07,2020-05-07 23:40:30,2020-10-19 01:08:31,맛,보통이에요,포장,꼼꼼해요,맛은 맛있는데 짠것도 많아요,gks*******,78935758
4,172251938,537157714,5,5,11_하프라이스 현미야채 1팩_SM17,2020-10-15 12:41:09,2020-10-18 21:31:13,맛,맛있어요,포장,꼼꼼해요,소스에 볶음밥 해먹으려 시켷어용,seh********,64853450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461185,164496338,4376621606,5,5,02. 비비고 왕교자 420g x 5개,2020-09-16 23:31:24,2020-09-18 20:39:45,맛,맛있어요,포장,꼼꼼해요,완전 저렴하게 구매 했습니다,lh3***,39539221
461186,164480242,4376621606,5,4,02. 비비고 왕교자 420g x 5개,2020-09-16 21:55:15,2020-09-18 19:54:22,맛,맛있어요,포장,꼼꼼해요,맛있고 배송도 빠르고 굿입니당~~^^♡,kak*****,2196078
461187,164472414,4376621606,4,4,02. 비비고 왕교자 420g x 5개,2020-09-16 23:13:01,2020-09-18 19:19:18,맛,맛있어요,포장,꼼꼼해요,유통기한도 길고 아이스팩도 넉넉히 넣어주시고 감사합니다,asd**,3899921
461188,164463954,4376621606,5,4,02. 비비고 왕교자 420g x 5개,2020-09-16 23:21:58,2020-09-18 18:37:27,맛,맛있어요,포장,꼼꼼해요,살짝 녹아서 왔지만 바로 냉동실로 옮겼네요^^;; 저렴하게 잘 샀습니다~,agl*****,50873081


In [7]:
error_df = pd.read_json('error_page_간편식_냉장_냉동_수정.json')
error_df

,0
0,http://www.tmon.co.kr/api/detail/direct/v1/det...
1,http://www.tmon.co.kr/api/detail/direct/v1/det...
2,http://www.tmon.co.kr/api/detail/direct/v1/det...
3,http://www.tmon.co.kr/api/detail/direct/v1/det...
4,http://www.tmon.co.kr/api/detail/direct/v1/det...
...,...
70,http://www.tmon.co.kr/api/detail/direct/v1/det...
71,http://www.tmon.co.kr/api/detail/direct/v1/det...
72,http://www.tmon.co.kr/api/detail/direct/v1/det...
73,http://www.tmon.co.kr/api/detail/direct/v1/det...
